A ideia desse algoritmo é classificar textos em portugues como positivo, negativo ou neutro usando aprendizado de máquina supervisionado

a base de dados usadas para treinar o modelo está disponiveil a seguir:
[Teets_MG.csv](https://www.kaggle.com/leandrodoze/tweets-from-mgbr)

In [104]:
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import spacy
from spacy import displacy
import string 
import os
import gc


In [81]:
df = pd.read_csv('/home/renato/Documentos/Datasets//Tweets_Mg.csv')

A base contem alguns tweets  classificados como positivo, negativo e neutro que sera usada  para treinar o modelo

In [84]:
# arrumando a base de dados
rm_df = [x for x in list(df.columns) if x not in ['Text','Classificacao']]
df = df.drop(columns=rm_df)

df.head()


,Text,Classificacao
0,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro
1,"� @ Governador Valadares, Minas Gerais https:/...",Neutro
2,"�� @ Governador Valadares, Minas Gerais https:...",Neutro
3,��� https://t.co/BnDsO34qK0,Neutro
4,��� PSOL vai questionar aumento de vereadores ...,Negativo


In [85]:
df.columns = ['Text', 'Polaridade']

In [86]:
df_concat['Polaridade'].value_counts()


Positivo    27994
Negativo    27211
Neutro       2453
Name: Polaridade, dtype: int64

Criando o modelo usando naive bayes

In [87]:
tweets = df["Text"].values
classes = df["Polaridade"].values
classes

array(['Neutro', 'Neutro', 'Neutro', ..., 'Positivo', 'Positivo',
       'Positivo'], dtype=object)

Agora, vamos treinar o modelo usando a abordagem Bag of Words e o algoritmo Naive Bayes Multinomial 
 - Bag of Words, na prática, cria um vetor com cada uma das palavras do texto completo da base, depois, calcula a frequência em que essas palavras ocorrem em uma data sentença, para então classificar/treinar o modelo
- Exemplo HIPOTÉTICO de três sentenças vetorizadas "por palavra" e classificadas baseada na frequência de suas palavras:
 {0,3,2,0,0,1,0,0,0,1, Positivo}
 {0,0,1,0,0,1,0,1,0,0, Negativo}
 {0,1,1,0,0,1,0,0,0,0, Neutro}
 - Olhando para esses vetores, meu palpite é que as palavras nas posições 2 e 3 são as com maior peso na determinação de a que classe pertence cada uma das três sentenças avaliadas
 - A função fit_transform faz exatamente esse processo: ajusta o modelo, aprende o vocabulário, e transforma os dados de treinamento em feature vectors, a.k.a. vetor com frequêcia das palavras

### Usando Naive Bayes

In [88]:
vectorizer = CountVectorizer(analyzer = "word")
freq_tweets = vectorizer.fit_transform(tweets)

modelo = MultinomialNB()
modelo.fit(freq_tweets, classes)

MultinomialNB()

In [89]:
# testando modelo com alguns tweets retirados do meu feed
testes = ["Qual é o melhor jeito de aumentar a renda no Brasil?", "Legalizar o jogo do bicho", "Eu pensei que seria um aniversário feliz. Tá sendo não :("]

freq_testes = vectorizer.transform(testes)
predict = modelo.predict(freq_testes)

for i in range(len(testes)):
    print(f"{testes[i]} - {predict[i]}")

Qual é o melhor jeito de aumentar a renda no Brasil? - Neutro
Legalizar o jogo do bicho - Neutro
Eu pensei que seria um aniversário feliz. Tá sendo não :( - Neutro


usando validação cruzada para medir a acuracia do modelo


In [90]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv = 10)
metrics.accuracy_score(classes, resultados)

0.8831564824978656

### Testando analise de sentimento com random forest

In [92]:
vectorizer = CountVectorizer(analyzer = "word")
freq_tweets = vectorizer.fit_transform(tweets)

modelo = RandomForestClassifier()
modelo.fit(freq_tweets, classes)

RandomForestClassifier()

In [93]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv = 10)
metrics.accuracy_score(classes, resultados)

0.889132821075741

### usando TF-IDF ao inves do CountVectorizer

In [96]:
X = df['Text']
y = df['Polaridade']

In [105]:

punctuations = string.punctuation
def spacy_tokenizer(sentence):
    sentence = sentence.strip().lower()
    mytokens = nlp(sentence)
    mytokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens]
    mytokens = [word for word in  mytokens if word not in pt_spacy_stopwords and word not in punctuations]

    return mytokens


In [111]:
nlp = spacy.load('pt_core_news_sm')
pt_spacy_stopwords = spacy.lang.pt.stop_words.STOP_WORDS

In [106]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=spacy_tokenizer)

In [107]:

X, _, y, _ = train_test_split(X, y, test_size=0.90)


In [112]:
X_tfidf =tfidf_vectorizer.fit_transform(X)
X_tfidf.shape


(81, 485)

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.30)

In [115]:
classifier = RandomForestClassifier()

In [117]:
classifier.fit(X_train, y_train)

RandomForestClassifier()

In [123]:
resultados = cross_val_predict(classifier, X_train, y_train, cv = 10)
metrics.accuracy_score(y_train, resultados)

0.75